In [40]:
import pandas as pd
import numpy as np
from math import sqrt

In [ ]:
# 定義欄位名稱
column_names = ['age','menopause','tumor-size','inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat','class']  
# 讀取 txt 檔案，並轉為 DataFrame 格式
df = pd.read_csv("breast-cancer.txt",names = column_names)
df.head(3)


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'


In [ ]:
# 計算 feature entropy
def entropy(df,feature):
    prob = df[feature].value_counts(normalize = True)
    return -np.sum(prob * np.log2(prob))

# 計算 Mutual Information
def mutual_information(X, Y):
    # 計算 X 和 Y 的熵
    H_X = entropy(X)
    H_Y = entropy(Y)
    # 計算 X 和 Y 的聯合機率
    joint_prob = pd.crosstab(X, Y, normalize=True)
    mi = -np.sum(joint_prob * np.log2(joint_prob))
    return mi

# 計算特徵 X、Y 的 symmetric uncertainty
def cal_su(X,Y):
    H_X = entropy(X)
    H_Y = entropy(Y)
    return 2 * (mutual_information(X,Y) / (H_X + H_Y))

# 計算選取的特徵子集對於類別預測的 Goodness
def Goodness(feature_subset,label):
    su_X_C = 0
    sum_su_X_Y = 0
    for feature in feature_subset:
        su_X_C += cal_su(feature,label)
    for feature_i in feature_subset:
        for feature_j in feature_subset:
            sum_su_X_Y += cal_su(feature_i,feature_j)
    return su_X_C / sqrt(sum_su_X_Y)



forward selection

In [ ]:
# X 代表資料集的特徵集合，y 則是類別值
def forward_selection(X, y):
    select_features = []
    best_score = 0
    remaining_features = list(range(X.shpae(1)))
    while len(remaining_features) > 0:
        scores = []
        for feature in remaining_features:
            temp_features = select_features + [feature]
            score = Goodness(temp_features,y)
            scores.append((temp_features,scores))
        
        scores.sort(reverse = True)
        best_new_features,best_new_score = scores[0]

        if best_new_score > best_score:
            best_score = best_new_score

9
